In [2]:
import json
import requests
import csv
import pandas as pd
import openweathermapy.core as ow
import matplotlib.pyplot as plt
from config import *
from citipy import citipy
from geopy.geocoders import Nominatim
from geopy.distance import vincenty, great_circle
import random
import time

owm_settings = {"units": "imperial", "appid": OWM_KEY}

In [3]:
#CSV contains ISO 3166-1 alpha-2 country codes
#Necessary to convert country codes provided by citipy to country names for compatibility with geopy.
#geopy does not recognize all two-letter country codes
reader = csv.reader(open('countrycodes.csv', 'r'))
country_code_dict = dict(reader)

In [5]:
#Load OWM city list json file
owm_city_json = json.load(open('city.list.json', encoding='utf8'))

In [7]:
#WORKING#

city_count = 10
counter = 0

rangeLat = (-90, 90)
rangeLong = (-180, 180)

latitudes = []
longitudes = []

cities = []

geolocator = Nominatim()
start_time = time.time()
while counter != city_count:
    rand_lat = round(random.uniform(*rangeLat), 5)
    rand_long = round(random.uniform(*rangeLong), 5)
    rand_coord = (rand_lat, rand_long)
    city = citipy.nearest_city(rand_lat, rand_long)
    name = city.city_name
    country = city.country_code
    
    city_coord = ''
    for x in owm_city_json:
        if x['name'] == name.title() and x['country'] == country.upper():
            city_coord = (x['coord']['lat'], x['coord']['lon'])
            owm_id = x['id']
            
    if city_coord != '':
        try: #citipy does not use special characters in city names which results in an attribute error from geopy as loc = None
            distance = vincenty(rand_coord, city_coord).miles #using vincenty since it's more accurate than great_circle, errors when handlign antipodes is not an issue here since the closest city to any given coord will not be an antipode
            if distance <= 69: #at equator, each line of latitude is approximately 69 miles from the next line, same for longitude.  Eliminating random points that have no nearby city within one degree of lat/long
                cities.append(owm_id)
                counter = counter + 1
        except AttributeError:        
            continue
            
print(time.time() - start_time)
print(cities)

4.656946420669556
[5993072, 2815392, 4330236, 3455020, 498711, 2023584, 2253354, 3166509, 6162654, 5026114]


In [14]:
city_count = 10
counter = 0

rangeLat = (-90, 90)
rangeLong = (-180, 180)

latitudes = []
longitudes = []

cities = []

geolocator = Nominatim()
start_time = time.time()
while counter != city_count:
    rand_lat = round(random.uniform(*rangeLat), 5)
    rand_long = round(random.uniform(*rangeLong), 5)
    rand_coord = (rand_lat, rand_long)
    city = citipy.nearest_city(rand_lat, rand_long)
    name = city.city_name
    country = country_code_dict[city.country_code]
    loc = geolocator.geocode(f'{name}, {country}')
    try: #citipy does not use special characters in city names which results in an attribute error from geopy as loc = None
        city_coord = (loc.latitude, loc.longitude)
        distance = vincenty(rand_coord, city_coord).miles #using vincenty since it's more accurate than great_circle, errors when handlign antipodes is not an issue here since the closest city to any given coord will not be an antipode
        if distance <= 69: #at equator, each line of latitude is approximately 69 miles from the next line, same for longitude.  Eliminating random points that have no nearby city within one degree of lat/long
            cities.append(name)
            counter = counter + 1
    except AttributeError:        
        continue
print(time.time() - start_time)
print(cities)

GeocoderTimedOut: Service timed out

In [ ]:
#####BIG IDEA.....WOAH!!!####
#Since the geocode module is timing out while making API calls to Nominatim, use OWM city.list.json instead
#city.list.json contains lat/long for each city name which cna be returned for the distance check
#also, can simultaneously verify existence of city in the OWM list

In [33]:
city_count = 5
counter = 0

rangeLat = (-90, 90)
rangeLong = (-180, 180)

latitudes = []
longitudes = []

cities = []

geolocator = Nominatim()
start_time = time.time()

while counter != city_count:
    rand_lat = round(random.uniform(*rangeLat), 5)
    rand_long = round(random.uniform(*rangeLong), 5)
    rand_coord = (rand_lat, rand_long)
    city = citipy.nearest_city(rand_lat, rand_long)
    name = city.city_name
    country = country_code_dict[city.country_code]
    time.sleep(1.1)
    loc = geolocator.geocode(f'{name}, {country}')
    try: #citipy does not use special characters in city names which results in an attribute error from geopy as loc = None
        city_coord = (loc.latitude, loc.longitude)
        distance = vincenty(rand_coord, city_coord).miles #using vincenty since it's more accurate than great_circle, errors when handlign antipodes is not an issue here since the closest city to any given coord will not be an antipode
        if distance <= 69: #at equator, each line of latitude is approximately 69 miles from the next line, same for longitude.  Eliminating random points that have no nearby city within one degree of lat/long
            cities.append(name)
            counter = counter + 1
    except AttributeError:        
        continue
   
        
print(time.time() - start_time)
#print(cities)

GeocoderTimedOut: Service timed out

In [32]:
print(cities)
print(name)
print(country)

#loc = geolocator.geocode("dalbandin, pakistan")
#print(loc)

['chongwe', 'juneau', 'tapes', 'chaohu', 'thanh hoa']
thanh hoa
Viet Nam


In [34]:
#print(cities)
#['iqaluit', 'worland', 'mlonggo', 'bayir', 'college', 'choix', 'the pas', 'tlacotalpan', 'glace bay', 'port jervis']

#NOTE: Need to compare cities to the openweathermap city.list.json when pulling cities
#Only return cities in the OWM list
#For example, "bayir" returns no city from OWM

#NOTE: May also need to figure out a way to handle cities like "college" which have multiple macthes in the OWM list

#NOTE: Could also pull the city 'id' from OWM for use with API later (not necessary though)

#NOTE: Once city list is generated, write to csv to save names and avoid having to call citipy/geopy when re-running

#WARNING: Must solve for "GeocoderTimedOut: Service timed out"

#Time w/ great_circle: 41, 53, 66 seconds

In [26]:
geolocator = Nominatim()

name = 'bryan'
country = 'us'
coord = (36.002, -90)

location = geolocator.geocode(f'{name}, {country}')
city_coord = (location.latitude, location.longitude)

dist = vincenty(coord, city_coord).miles

print(dist)

274.63112998109415


In [22]:
test = 10
count = 0

while count != test:
    print(count)
    count = count + 1

0
1
2
3
4
5
6
7
8
9
